In [1]:
from langchain.llms import OpenAI

In [2]:
import os 
os.environ["OPENAI_API_KEY"]="sh-SdFOSWYfLyYweLXB4uu7T3BlbkFJtY2XYzNFTBUfGf6Z0000"

In [3]:
llm=OpenAI(temperature=0.6)

In [4]:
text="What is the capital of England"
print(llm.predict(text))



The capital of England is London.


In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="hf_vryLwlkxIojKwWHhXEJaDCtSz"


In [6]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

c:\Users\rahul\End to End LLM Langchain Project\langenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rahul\End to End LLM Langchain Project\langenv\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [7]:
output=llm_huggingface.predict("Can you tell me the capital of poland")
print(output)

Warsaw


In [8]:
from langchain.prompts import PromptTemplate
prompt_template=PromptTemplate(input_variables=['country'],
                               template="Tell me the capital of {country}")

prompt_template.format(country="India")

'Tell me the capital of India'

In [9]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm,prompt=prompt_template)
print(chain.run("Sweden"))



The capital of Sweden is Stockholm.


## Combining Multiple Chains using Simple Sequential Chain

In [10]:
capital_prompt=PromptTemplate(input_variables=['country'],
                              template="What is the capital of {country}")
capital_chain=LLMChain(llm=llm,prompt=capital_prompt)

famous_template=PromptTemplate(input_variables=['capital'],
                               template="Tell me about some of the popular places to visit in {capital}")
famous_chain=LLMChain(llm=llm,prompt=famous_template)


## Constructing the LLM Chain

In [11]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("England")

" It is a bustling city with a rich history and a diverse culture. There are many popular places to visit in London, including:\n\n1. Buckingham Palace - The official residence of the British monarch, this iconic palace is a must-see for visitors. You can take a tour of the State Rooms and see the famous Changing of the Guard ceremony.\n\n2. Big Ben and the Houses of Parliament - Located on the banks of the River Thames, these historic buildings are a symbol of London and its political history.\n\n3. The London Eye - This giant Ferris wheel offers stunning views of the city skyline and is a popular attraction for tourists.\n\n4. Tower of London - This medieval castle has served as a royal palace, prison, and execution site. Visitors can see the Crown Jewels and explore the tower's dark history.\n\n5. The British Museum - One of the world's largest and most comprehensive museums, it houses a vast collection of artifacts from around the world, including the famous Rosetta Stone.\n\n6. Hy

## Sequential Chain

In [12]:
capital_prompt=PromptTemplate(input_variables=['country'],
                              template="What is the capital of {country}")
capital_chain=LLMChain(llm=llm,prompt=capital_prompt,output_key="capital")

In [13]:
famous_template=PromptTemplate(input_variables=['capital'],
                               template="Tell me about some of the popular places to visit in {capital}")
famous_chain=LLMChain(llm=llm,prompt=famous_template,output_key="places")

In [14]:
from langchain.chains import SequentialChain

In [15]:
chain=SequentialChain(chains=[capital_chain,famous_chain],
                      input_variables=['country'],
                      output_variables=['capital','places'])
chain({'country':'Italy'})


{'country': 'Italy',
 'capital': '\n\nThe capital of Italy is Rome.',
 'places': " \n\nSome popular places to visit in Rome include the Colosseum, an ancient Roman amphitheater that is a symbol of the city's rich history; the Vatican City, the headquarters of the Roman Catholic Church and home to famous landmarks such as St. Peter's Basilica and the Sistine Chapel; the Pantheon, a well-preserved ancient temple dedicated to the Roman gods; and the Trevi Fountain, a stunning Baroque fountain where visitors can toss a coin and make a wish.\n\nOther popular places in Italy include Florence, known for its Renaissance art and architecture, including the famous Duomo cathedral and the Uffizi Gallery; Venice, a beautiful city built on water with iconic landmarks such as St. Mark's Basilica and the Grand Canal; and the Amalfi Coast, a stunning stretch of coastline dotted with picturesque towns and villages.\n\nIn northern Italy, the city of Milan is a popular destination for fashion and design 

## Using Langchain and Prompt Chains to extract most common disease of the Liver with its ICD 10 code

In [16]:
disease_prompt=PromptTemplate(input_variables=['organ'],
                              template="What is the most common disease of the {organ}")
disease_chain=LLMChain(llm=llm,prompt=disease_prompt,output_key="disease")

In [17]:
icd_template=PromptTemplate(input_variables=['disease'],
                               template="ICD 10 code of the {disease}")
icd_chain=LLMChain(llm=llm,prompt=icd_template,output_key="icd 10 code")

In [18]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[disease_chain,icd_chain],
                      input_variables=['organ'],
                      output_variables=['disease','icd 10 code'])
a=chain({'organ':'liver'})

## Using Regex to extract the ICD 10 Codes for the Disease

In [19]:
import re

def extract_icd10_codes(text):
    # ICD-10 codes pattern: Letter followed by two digits, and an optional dot with up to 4 alphanumeric characters
    pattern = r'\b[A-Z][0-9]{2}(?:\.[A-Z0-9]{1,4})?\b'
    
    # Find all matches in the text
    matches = re.findall(pattern, text)
    
    return matches

codes = extract_icd10_codes(a['icd 10 code'])
print("ICD-10 Codes for the most common Liver disease:", codes)

ICD-10 Codes for the most common Liver disease: ['K76.0', 'B18', 'K70', 'K74', 'C22']


## Chat Models with Chat OpenAI

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [22]:
chat_llm=ChatOpenAI(temperature=0.6,model='gpt-3.5-turbo')

In [23]:
chat_llm([
    SystemMessage(content="You are a Medical AI assistant"),
    HumanMessage(content="Please give some advice on management of Liver Cirrhosis")
])

AIMessage(content="I'm not a doctor, but I can provide some general advice on managing liver cirrhosis. It's important to consult with a healthcare professional for personalized guidance. Here are a few tips:\n\n1. Medication management: Follow your doctor's prescribed medications strictly, including those for underlying conditions like hepatitis B or C, and avoid medications that may harm the liver, such as certain painkillers or sedatives.\n\n2. Maintain a healthy diet: Consume a well-balanced diet low in sodium, fat, and refined sugars. Focus on fresh fruits, vegetables, whole grains, lean proteins, and healthy fats. Limit alcohol consumption or avoid it completely, as it can worsen liver damage.\n\n3. Manage fluid retention: Limit your sodium intake to help reduce fluid retention. Your doctor may also recommend diuretic medications to help remove excess fluid from the body.\n\n4. Vaccinations: Ensure you are up to date on vaccinations, especially for hepatitis A and B, as these inf

## Prompt Template + LLM + Output Parsers

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser 

In [59]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split("\n")[1:]

In [60]:
template="You are a helpful medical assistant.When a user gives an input of a disease, generate the names of 5 drugs used to treat the disease in a comma seperated list."
human_template="{disease}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [61]:
chain=chatprompt|chat_llm|Commaseperatedoutput()

In [63]:
chain.invoke({"disease":"cirrhosis"})

['1. Ursodiol',
 '2. Lactulose',
 '3. Spironolactone',
 '4. Furosemide',
 '5. Propranolol']